In [1]:
import os
import pandas as pd

In [2]:
quadAlpha = '/media/ak/My Passport/QuadAlpha'

In [3]:
file = 'XM1_tick_new_dictionary.pkl'

In [4]:
fileLoc = os.path.join(quadAlpha, file)

In [5]:
fileLoc

'/media/ak/My Passport/QuadAlpha/XM1_tick_new_dictionary.pkl'

In [7]:
dd = pd.read_pickle(fileLoc)

In [22]:
def remove_nested_empty_dicts(dictionary):
    if not isinstance(dictionary, dict):
        return dictionary
    else:
        new_dict = {}
        for key, value in dictionary.items():
            if isinstance(value, dict):
                value = remove_nested_empty_dicts(value)
            if value:
                new_dict[key] = value
        return new_dict
    
import pandas as pd
import ast

def flatten_dict(d):
    def expand(key, value):
        if isinstance(value, dict):
            return [(str(key) + '_' + str(k), v) for k, v in flatten_dict(value).items()]
        else:
            return [(str(key), value)]
    
    items = [item for k, v in d.items() for item in expand(k, v)]
    
    return dict(items)

def dict_to_df(d):
    # Flatten the dictionary
    flat_dict = flatten_dict(d)

    # Create a DataFrame and transpose it for correct orientation
    df = pd.DataFrame(flat_dict, index=[0]).T

    # Remove unwanted text from column names
    df.reset_index(inplace=True)
    df.columns = ['keys', 'values']
    df['keys'] = df['keys'].str.replace('results_\d_', '')

    # Split the keys into tuple key and variable name
    df[['tuple_key', 'variable']] = df['keys'].str.rsplit('_', n=1, expand=True)
    
    # Convert string representation of tuple to tuple
    df['tuple_key'] = df['tuple_key'].apply(ast.literal_eval)

    # Split the tuple keys into individual columns
    df[['start', 'end', 'shift', 'window']] = pd.DataFrame(df['tuple_key'].tolist(), index=df.index)

    # Drop the tuple_key column
    df.drop(columns=['tuple_key'], inplace=True)

    return df



In [19]:
non_empty_dd = remove_nested_empty_dicts(dd)

In [34]:
non_empty_dd

{(0,
  2,
  2,
  35): {(0, 2, 2, 35): {'results_0_perm_mmds1': -0.005932339744891768,
   'results_0_chi2_weights': 4.5536884936937453e-20,
   'results_0_sim_mmds': -0.0030805050277622867,
   'results_0_sig2': 0.44520779505128716,
   'results_0_Kxy': 0.6571426348218667,
   'results_0_mean': 0.006134763223032191,
   'results_0_var': 6.709856332479955e-08,
   'results_0_Kxx': 0.6390709454988164,
   'results_0_Kyy': 0.6759885763999878,
   'results_0_mean_gram': 0.006134763223032191,
   'results_0_var_gram': 6.709856332479955e-08,
   'results_0_med': 0.44520779505128716,
   'results_0_powers': 0.3578282269072021,
   'results_0_best_ker': 'KGauss(w2=0.012)',
   'results_0_XZ_test_alpha': 0.05,
   'results_0_XZ_test_pvalue': 0.005,
   'results_0_XZ_test_test_stat': 0.05172463945606531,
   'results_0_XZ_test_h0_rejected': True,
   'results_0_XZ_test_list_permuted_mmd2': -0.0029607032892886853,
   'results_0_YZ_test_alpha': 0.05,
   'results_0_YZ_test_test_stat': 0.09497770884909096,
   'result

In [35]:
import pandas as pd
import re

def flatten_dict(d):
    items = []
    for k, v in d.items():
        if isinstance(v, dict):
            for k2, v2 in v.items():
                if isinstance(v2, dict):
                    for k3, v3 in v2.items():
                        new_key = re.sub('results_\d_', '', k3)
                        items.append((*k, new_key, v3))
    return items

def dict_to_df(d):
    # Flatten the dictionary
    flat_items = flatten_dict(d)

    # Create a DataFrame and split tuple keys into separate columns
    df = pd.DataFrame(flat_items, columns=['start', 'end', 'shift', 'window', 'variables', 'values'])

    # Pivot the DataFrame to have variables as columns
    df = df.pivot_table(index=['start', 'end', 'shift', 'window'], columns='variables', values='values')

    return df

In [40]:
non_empty_dd[(5, 10, 5, 5)][(5, 10, 5, 5)].keys()

dict_keys(['results_5_perm_mmds1', 'results_5_chi2_weights', 'results_5_sim_mmds', 'results_5_sig2', 'results_5_Kxy', 'results_5_mean', 'results_5_var', 'results_5_Kxx', 'results_5_Kyy', 'results_5_mean_gram', 'results_5_var_gram', 'results_5_med', 'results_5_powers', 'results_5_best_ker', 'results_5_XZ_test_alpha', 'results_5_XZ_test_test_stat', 'results_5_XZ_test_h0_rejected', 'results_5_XZ_test_list_permuted_mmd2', 'results_5_YZ_test_alpha', 'results_5_YZ_test_test_stat', 'results_5_YZ_test_h0_rejected', 'results_5_YZ_test_list_permuted_mmd2', 'start_point', 'end_point', 'window'])

In [51]:
import pandas as pd
def dict_to_df(input_dict):
    # Initialize an empty list to store the data
    data = []
    # Iterate over the outer dictionary
    for outer_value in input_dict.values():
        # Iterate over the nested dictionaries
        for inner_key, values in outer_value.items():
            # Split the inner key and add it to the dictionary values
            row = {'key1': inner_key[0], 'key2': inner_key[1], 'key3': inner_key[2], 'key4': inner_key[3]}
            row.update(values)
            data.append(row)
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    # Set the keys as the index
    df.set_index(['key1', 'key2', 'key3', 'key4'], inplace=True)
    # Remove "results_0_" from column names
    df.columns = df.columns.str.replace('results_', '')
    return df


# def dict_to_df(input_dict):
#     # Initialize an empty list to store the data
#     data = []
#     # Iterate over the outer dictionary
#     for outer_value in input_dict.values():
#         # Iterate over the nested dictionaries
#         for inner_key, values in outer_value.items():
#             # Add the inner key to the dictionary values
#             row = {'inner_key': inner_key}
#             row.update(values)
#             data.append(row)
#     # Convert the list of dictionaries to a DataFrame
#     df = pd.DataFrame(data)
#     # Set the key as the index
#     df.set_index('inner_key', inplace=True)
#     # Remove "results_0_" from column names
#     df.columns = df.columns.str.replace('results_0_', '')
#     return df
# def dict_to_df(input_dict):
#     # Initialize an empty list to store the data
#     data = []
#     # Iterate over the outer dictionary
#     for outer_value in input_dict.values():
#         # Iterate over the nested dictionaries
#         for inner_key, values in outer_value.items():
#             # Split the inner key and add it to the dictionary values
#             row = {'key1': inner_key[0], 'key2': inner_key[1], 'key3': inner_key[2], 'key4': inner_key[3]}
#             row.update(values)
#             data.append(row)
#     # Convert the list of dictionaries to a DataFrame
#     df = pd.DataFrame(data)
#     # Set the keys as the index
#     df.set_index(['key1', 'key2', 'key3', 'key4'], inplace=True)
#     # Remove "results_0_" from column names
#     df.columns = df.columns.str.replace('results_0_', '')
#     return df


# Your nested dictionary
nested_dict = {
    (0, 2, 2, 35): {(0, 2, 2, 35): {'results_0_perm_mmds1': -0.005932339744891768, 
                                     'results_0_chi2_weights': 4.5536884936937453e-20, 
                                     'results_0_sim_mmds': -0.0030805050277622867, 
                                     'results_0_sig2': 0.44520779505128716, 
                                     'results_0_Kxy': 0.6571426348218667, 
                                     'results_0_mean': 0.006134763223032191, 
                                     'results_0_var': 6.709856332479955e-08, 
                                     'results_0_Kxx': 0.6390709454988164, 
                                     'results_0_Kyy': 0.6759885763999878, 
                                     'results_0_mean_gram': 0.006134763223032191, 
                                     'results_0_var_gram': 6.709856332479955e-08, 
                                     'results_0_med': 0.44520779505128716, 
                                     'results_0_powers': 0.3578282269072021, 
                                     'results_0_best_ker': 'KGauss(w2=0.012)', 
                                     'results_0_XZ_test_alpha': 0.05, 
                                     'results_0_XZ_test_pvalue': 0.005, 
                                     'results_0_XZ_test_test_stat': 0.05172463945606531, 
                                     'results_0_XZ_test_h0_rejected': True, 
                                     'results_0_XZ_test_list_permuted_mmd2': -0.0029607032892886853, 
                                     'results_0_YZ_test_alpha': 0.05, 
                                     'results_0_YZ_test_test_stat': 0.09497770884909096, 
                                     'results_0_YZ_test_h0_rejected': True, 
                                     'results_0_YZ_test_list_permuted_mmd2': -0.002814597364055746, 
                                     'end_point': 2, 
                                     'window': 35}}
}




In [57]:
df = dict_to_df(non_empty_dd)


In [64]:
#cols_to_rename = ['XZ_test_pvalue', 'YZ_test_pvalue', 'XZ_test_list_permuted_mmd2', 'YZ_test_list_permuted_mmd2']
cols_to_rename = ['YZ_test_alpha',
 'YZ_test_test_stat',
 'YZ_test_h0_rejected', 'YZ_test_list_permuted_mmd2', 'XZ_test_list_permuted_mmd2', 'XZ_test_test_stat']

# Loop over columns in dataframe
for col in df.columns:
    # Loop over the strings in cols_to_rename
    for string in cols_to_rename:
        # If the string is in the column name
        if string in col:
            # Replace everything before (and including) the last underscore
            new_name = col.split(string)[1]
            df = df.rename(columns={col: string + new_name})

df

0_perm_mmds1  0_chi2_weights  0_sim_mmds    0_sig2  \
key1 key2 key3 key4                                                       
0    2    2    35       -0.005932    4.553688e-20   -0.003081  0.445208   
     1    1    45       -0.005244    9.258896e-21   -0.002751  0.325785   
               75       -0.005244    9.258896e-21   -0.002751  0.325785   
               65       -0.005244    9.258896e-21   -0.002751  0.325785   
               5        -0.005244    9.258896e-21   -0.002751  0.325785   
...                           ...             ...         ...       ...   
130  131  1    5              NaN             NaN         NaN       NaN   
          4    5              NaN             NaN         NaN       NaN   
129  130  5    5              NaN             NaN         NaN       NaN   
     131  3    5              NaN             NaN         NaN       NaN   
          2    5              NaN             NaN         NaN       NaN   

                        0_Kxy    0_mean         0_var     0_Kxx     0_Kyy  \
key1 key2 key3 key4                                                         
0    2    2    35    0.657143  0.006135  6.709856e-08  0.639071  0.675989   
     1    1    45    0.733375  0.017321  3.066185e-06  0.763567  0.717826   
               75    0.733375  0.017321  3.066185e-06  0.763567  0.717826   
               65    0.733375  0.017321  3.066185e-06  0.763567  0.717826   
               5     0.733375  0.017321  3.066185e-06  0.763567  0.717826   
...                       ...       ...           ...       ...       ...   
130  131  1    5          NaN       NaN           NaN       NaN       NaN   
          4    5          NaN       NaN           NaN       NaN       NaN   
129  130  5    5          NaN       NaN           NaN       NaN       NaN   
     131  3    5          NaN       NaN           NaN       NaN       NaN   
          2    5          NaN       NaN           NaN       NaN       NaN   

                     0_mean_gram  ...  XZ_test_test_stat  \
key1 key2 key3 key4               ...                      
0    2    2    35       0.006135  ...                NaN   
     1    1    45       0.017321  ...                NaN   
               75       0.017321  ...                NaN   
               65       0.017321  ...                NaN   
               5        0.017321  ...                NaN   
...                          ...  ...                ...   
130  131  1    5             NaN  ...           0.385982   
          4    5             NaN  ...           0.376894   
129  130  5    5             NaN  ...                NaN   
     131  3    5             NaN  ...                NaN   
          2    5             NaN  ...                NaN   

                     130_XZ_test_h0_rejected  XZ_test_list_permuted_mmd2  \
key1 key2 key3 key4                                                        
0    2    2    35                        NaN                         NaN   
     1    1    45                        NaN                         NaN   
               75                        NaN                         NaN   
               65                        NaN                         NaN   
               5                         NaN                         NaN   
...                                      ...                         ...   
130  131  1    5                        True                   -0.002756   
          4    5                        True                   -0.002453   
129  130  5    5                         NaN                         NaN   
     131  3    5                         NaN                         NaN   
          2    5                         NaN                         NaN   

                    YZ_test_alpha  YZ_test_test_stat  YZ_test_h0_rejected  \
key1 key2 key3 key4                                                         
0    2    2    35             NaN                NaN                  NaN   
     1    1    45             NaN         

In [63]:
[f for f in list(df.columns.values) if str('YZ') in f]

['0_YZ_test_alpha',
 '0_YZ_test_test_stat',
 '0_YZ_test_h0_rejected',
 'YZ_test_list_permuted_mmd2',
 'YZ_test_pvalue',
 '1_YZ_test_alpha',
 'YZ_test_pvalue',
 '1_YZ_test_test_stat',
 '1_YZ_test_h0_rejected',
 'YZ_test_list_permuted_mmd2',
 '2_YZ_test_alpha',
 '2_YZ_test_test_stat',
 '2_YZ_test_h0_rejected',
 'YZ_test_list_permuted_mmd2',
 'YZ_test_pvalue',
 '3_YZ_test_alpha',
 'YZ_test_pvalue',
 '3_YZ_test_test_stat',
 'YZ_test_list_permuted_mmd2',
 '3_YZ_test_h0_rejected',
 '4_YZ_test_alpha',
 '4_YZ_test_test_stat',
 '4_YZ_test_h0_rejected',
 'YZ_test_list_permuted_mmd2',
 'YZ_test_pvalue',
 '5_YZ_test_alpha',
 '5_YZ_test_test_stat',
 '5_YZ_test_h0_rejected',
 'YZ_test_list_permuted_mmd2',
 'YZ_test_pvalue',
 '6_YZ_test_alpha',
 '6_YZ_test_test_stat',
 '6_YZ_test_h0_rejected',
 'YZ_test_list_permuted_mmd2',
 'YZ_test_pvalue',
 '7_YZ_test_alpha',
 'YZ_test_pvalue',
 '7_YZ_test_test_stat',
 '7_YZ_test_h0_rejected',
 'YZ_test_list_permuted_mmd2',
 '8_YZ_test_alpha',
 'YZ_test_pvalue',
 

In [70]:
df_grouped_key4 = df.groupby(level='key1')

In [71]:
df_grouped_key4.mean()

,0_perm_mmds1,0_chi2_weights,0_sim_mmds,0_sig2,0_Kxy,0_mean,0_var,0_Kxx,0_Kyy,0_mean_gram,...,130_powers,130_XZ_test_alpha,XZ_test_test_stat,XZ_test_list_permuted_mmd2,YZ_test_alpha,YZ_test_test_stat,YZ_test_list_permuted_mmd2,YZ_test_pvalue,129_besti,YZ_test_pvalue
key1,,,,,,,,,,,,,,,,,,,,,
0,-0.005961,3.955778e-19,-0.003038,0.540393,0.604561,0.026676,0.000002,0.605442,0.609051,0.026676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
